In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.select import Select
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
from selenium.common.exceptions import TimeoutException
from selenium.common.exceptions import NoSuchElementException
from webdriver_manager.chrome import ChromeDriverManager
from time import sleep

class Equipe:
    def __init__(self):
        '''Possibilitar que acessemos os sites dentro do navegador Chrome.'''
        self.service = Service(ChromeDriverManager().install())
        self.options = webdriver.ChromeOptions()
        self.driver = webdriver.Chrome(service=self.service, options=self.options)
        self.driver.maximize_window()
        timeout = 60
        self.wait = WebDriverWait(self.driver, timeout)
    
    def acessar_performa(self, url, email, senha):
        '''Acessa a plataforma do Performa de acordo com o link passado, email e senha compartilhadas para entrar na conta.'''
        self.url = url
        self.email = email
        self.senha = senha
        
        self.driver.get(self.url)
        
        # esperar ate o elemento seja encontrado para fazer inserir as info de email e senha da conta
        try:
            elemento_esperado_email = self.wait.until(EC.visibility_of_element_located((
                By.XPATH, "//input[@id='Email']"
            )))

            email_input = self.driver.find_element(By.XPATH, "//input[@id='Email']")
            senha_input = self.driver.find_element(By.XPATH, "//input[@id='Senha']")
            entrar_button = self.driver.find_element(By.XPATH, "//button[@class='btn btn-bricky pull-right']")

            email_input.send_keys(self.email)
            senha_input.send_keys(self.senha)
            entrar_button.click()
        except TimeoutException:
            print("O elemento não foi encontrado no tempo especificado!")
    
    def home(self):
        '''Voltar para a página principal'''
        self.driver.get("https://performaqca.seven.adv.br/AcessoCorrespondente/main")
        sleep(3)

    def aguardando_aceite(self, url_aceite):
        '''Possibilita entrar na pagina referente ao widget Solicitações Aguardando Aceite.'''
        self.url_aceite = url_aceite
        self.driver.get(self.url_aceite)
    
    def aguardando_checkin(self, url_checkin):
        '''Possibilita entrar na pagina referente ao widget Solicitações Aguardando CheckIn.'''
        self.url_checkin = url_checkin
        self.driver.get(self.url_checkin)

    def checar_solicitacao_aceite(self):
        '''Faz a checagem das solicitações. Verifica se realmente existe algum processo para o usuário dar aceite.'''
        try:
            elemento_esperado_texto = self.wait.until(EC.visibility_of_element_located((
                By.XPATH, "//div//ol//li[@class='active']"
            )))
            try:
                registro_localizado = self.driver.find_element(By.XPATH, "//div//tbody//tr//td[text()='Nenhum registro localizado']")
                registro_localizado = registro_localizado.text
                if registro_localizado == "Nenhum registro localizado":
                    voltar_pagina_inicial = self.driver.find_element(By.XPATH, "//div//ol[@class='breadcrumb']//li//a[text()='Home']")
                    voltar_pagina_inicial = voltar_pagina_inicial.get_attribute("href")
                    self.driver.get(voltar_pagina_inicial)
                    sleep(2)
                    return False
            except Exception as e:
                pass
        except TimeoutException:
            print("O elemento não foi encontrado no tempo especificado!")
        
        try:
            exibicao_resultado = self.driver.find_elements(By.XPATH, "//div[@class='pag col-md-12 space-top']//div//span")
            exibicao_resultado = exibicao_resultado[0].text
            exibicao_resultado = int(exibicao_resultado.split()[-1])
            if exibicao_resultado > 0:
                sleep(2)
                return True
        except Exception as e:
            pass
        except TimeoutException:
            print("O elemento não foi encontrado no tempo especificado!")
        return False

    def total_paginas(self):
        '''Calcula a quantidade de paginas que temos dentro dessas atividades'''
        exibicao_resultado = self.driver.find_elements(By.XPATH, "//div[@class='pag col-md-12 space-top']//div//span")
        exibicao_resultado = exibicao_resultado[0].text
        exibicao_resultado = int(exibicao_resultado.split()[-1])
        total_itens = exibicao_resultado
        itens_por_pagina = 25
        total_paginas = (total_itens + itens_por_pagina - 1) // itens_por_pagina
        return int(total_paginas)
    
    def buscar_links_processos(self):
        '''Percorre as paginas para pegar os links dos processos.'''
        checagem = self.checar_solicitacao_aceite()
        lista_links_servicos = []
        if checagem:
            # checar se temos apenas 1 pagina ou se temos > 1 pagina. Se tiver > 1 pagina, percorrer todas as paginas e extrair links (PASSAR POR ENQT)
            numero_paginas = self.total_paginas()
            if numero_paginas == 1:
                try:
                    servicos = self.driver.find_elements(By.XPATH, "//div//table//tbody//tr[contains(@class,'gridrow')]//td//a[@title='visualizar']")
                    for link_servico in servicos:
                        link = link_servico.get_attribute("href")
                        lista_links_servicos.append(link)
                except Exception as e:
                    print("Erro em buscar_links_processos: ", e)
                return lista_links_servicos
            elif numero_paginas > 1:
                # no momento do desenvolvimento nao temos o botao para passar para a proxima pagina e testar
                for pag in range(1, numero_paginas+1):
                    pagina_button = self.driver.find_elements(By.XPATH, f"//ul[@class='pagination ']//li//a[@href='{pag}']")[0]
                    pagina_button.click()
                    sleep(5)
                    try:
                        servicos = self.driver.find_elements(By.XPATH, "//div//table//tbody//tr[contains(@class,'gridrow')]//td//a[@title='visualizar']")
                        for link_servico in servicos:
                            link = link_servico.get_attribute("href")
                            lista_links_servicos.append(link)
                    except Exception as e:
                        print("Erro em buscar_links_processos: ", e)
                return lista_links_servicos
        return lista_links_servicos

    def aceitar_solicitacao(self):
        '''Executa a atividade de dar aceite'''
        lista_servicos = self.buscar_links_processos()
        # verifica se a lista de links possui links para entrar
        if lista_servicos is not None:
            for s in lista_servicos:
                self.driver.get(s)
                sleep(2)
                try:
                    elemento_esperado_acoes = self.wait.until(EC.visibility_of_element_located((
                        By.XPATH, "//div[@id='box-dadosprincipais']//div//div//div//div//div//button[@class='btn btn-acoes']"
                    )))
                    acoes_button = self.driver.find_element(By.XPATH, "//div[@id='box-dadosprincipais']//div//div//div//div//div//button[@class='btn btn-acoes dropdown-toggle']")
                    acoes_button.click()
                    try:
                        elemento_esperado_aceitar_solicitacao = self.wait.until(EC.visibility_of_element_located((
                            By.XPATH, "//ul[@class='dropdown-menu']//li//a[@id='buttonAceitarSolicitacao']"
                        )))
                        aceitar_solicitacao_button = self.driver.find_element(By.XPATH, "//ul[@class='dropdown-menu']//li//a[@id='buttonAceitarSolicitacao']")
                        aceitar_solicitacao_button.click()
                        sleep(3)
                        yes_button = self.driver.find_element(By.XPATH, "//div[@class='modal-content']//div[@class='modal-footer']//button[@class='bootboxSim btn btn-primary'][text() = 'Sim']")
                        yes_button.click()
                        sleep(2)
                    except TimeoutException:
                        print("O elemento não foi encontrado no tempo especificado!")
                    except Exception as e:
                        print("Erro em: ", e)
                except TimeoutException:
                    print("O elementro não foi encontrado no tempo especificado!")
                except Exception as e:
                    print("Erro em: ", e)
        # voltar para a primeira pagina
        self.home()

    def preencher_informacoes(self):
        '''Executa a atividade de preencher as informacoes do aguardando checkin'''
        lista_servicos = self.buscar_links_processos()
        # verifica se a lista de links possui links para entrar
        if lista_servicos is not None:
            for s in lista_servicos:
                self.driver.get(s)
                sleep(2)
                try:
                    elemento_esperado_acoes = self.wait.until(EC.visibility_of_element_located((
                        By.XPATH, "//div[@id='box-dadosprincipais']//div//div//div//div//div//button[@class='btn btn-acoes']"
                    )))
                    acoes_button = self.driver.find_element(By.XPATH, "//div[@id='box-dadosprincipais']//div//div//div//div//div//button[@class='btn btn-acoes dropdown-toggle']")
                    acoes_button.click()
                    caminho_editar = self.driver.find_element(By.XPATH, "//ul[@class='dropdown-menu']//li//a[@title='Docs. de Representação']")
                    caminho_preencher = self.driver.find_element(By.XPATH, "//ul[@class='dropdown-menu']//li//a[@title='Docs. de Representação']")
                    if caminho_editar.text == "Editar Docs. de Representação":
                        # passar para o proximo link
                        continue
                    elif caminho_preencher.text == "Solicitar Docs. de Representação":
                        caminho_preencher.click()
                        sleep(2)
                        #providenciar carta de preposicao? SIM
                        carta = self.driver.find_element(By.XPATH, "//div//label[2]//div//ins")
                        carta.click()
                        sleep(1)
                        #nome
                        nome = self.driver.find_element(By.XPATH, "//div//input[@id='NomeCompletoPreposto']")
                        nome.send_keys('GIULIA ARRUDA BORGES DE ANDRADE')
                        #rg
                        rg = self.driver.find_element(By.XPATH, "//div//input[@id='RGPreposto']")
                        rg.send_keys('10.684.866 SDS/PE')
                        #cpf
                        cpf = self.driver.find_element(By.XPATH, "//div//input[@id='CPFPreposto']")
                        cpf.send_keys('111.314.934-51')
                        #providenciar subestabelecimento - NAO
                        subestabelecimento = self.driver.find_element(By.XPATH, '//*[@id="docsRepresentacaoForm"]/div[4]/div/label[1]/div/ins')
                        subestabelecimento.click()
                        #observacoes
                        obs = self.driver.find_element(By.XPATH, "//div//div//textarea[@id='ObservacoesDocsRepresentacao']")
                        obs.send_keys('GIULIA ARRUDA BORGES DE ANDRADE RG 10.684.866 SDS/PE CPF 111.314.934-51 (81)991248648 giuliaandrade@queirozcavalcanti.adv.br')
                        #salvar
                        salvar = self.driver.find_element(By.XPATH, '//*[@id="dialog-modal"]/div/div/div[3]/button[1]')
                        salvar.click()
                        sleep(2)
                except TimeoutException:
                    print("O elementro não foi encontrado no tempo especificado!")
                except Exception as e:
                    print("Erro em: ", e)
        # voltar para a primeira pagina
        self.home()
    
    def fechar_chrome(self):
        self.driver.quit()

if __name__ == "__main__":
    app = Equipe()
    url = "https://performaqca.seven.adv.br/AcessoCorrespondente/main"
    email = "giuliaandrade@queirozcavalcanti.adv.br"
    senha = "prazo"

    app.acessar_performa(url=url, email=email, senha=senha)
    sleep(2)

    # aguardando aceite
    url_aceite = "https://performaqca.seven.adv.br/AcessoCorrespondente/solicitacao/myList?statusId=2&modoExibicao=2"
    app.aguardando_aceite(url_aceite=url_aceite)
    sleep(3)
    app.aceitar_solicitacao()
    sleep(3)

    # aguardando check in
    url_checkin = "https://performaqca.seven.adv.br/AcessoCorrespondente/solicitacao/myList?statusId=12&modoExibicao=2&operation=aguardandocheckin"
    app.aguardando_checkin(url_checkin=url_checkin)
    sleep(3)
    app.preencher_informacoes()
    sleep(3)

    app.home()
    sleep(3)
    app.fechar_chrome()
    print("O programa finalizou com sucesso!")
else:
    print("O sistema não conseguiu carregar adequadamente!")

NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=124.0.6367.119)
Stacktrace:
	GetHandleVerifier [0x00AAC213+48259]
	(No symbol) [0x00A3CB41]
	(No symbol) [0x00930A17]
	(No symbol) [0x0090E02B]
	(No symbol) [0x0099742E]
	(No symbol) [0x009A9476]
	(No symbol) [0x00990B36]
	(No symbol) [0x0096570D]
	(No symbol) [0x009662CD]
	GetHandleVerifier [0x00D666A3+2908435]
	GetHandleVerifier [0x00DA3CBB+3159851]
	GetHandleVerifier [0x00B451CB+674875]
	GetHandleVerifier [0x00B4B38C+699900]
	(No symbol) [0x00A46344]
	(No symbol) [0x00A42398]
	(No symbol) [0x00A4252C]
	(No symbol) [0x00A34CB0]
	BaseThreadInitThunk [0x755900C9+25]
	RtlGetAppContainerNamedObjectPath [0x77107B4E+286]
	RtlGetAppContainerNamedObjectPath [0x77107B1E+238]


In [ ]:
# PARA INSTALAR AS BIBLIOTECAS BASTA UTILIZAR O SINAL DE ! ANTES DO COMANDO pip instal {nome_da_lib}
# !pip install pandas selenium webdriver-manager openpyxl